In [14]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

In [3]:
path = '흥부와놀부.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

corpus length: 2210


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 311


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []


In [6]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 724
Vectorization...


In [7]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [9]:
def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [10]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])


Epoch 1/60
724/724 [==============================] - 6s 8ms/step - loss: 6.5175

----- Generating text after Epoch: 0
----- Generating with seed: "마리가 제비 둥지를 덮치려는 참이었습니다. 흥부는 얼른 구렁이를 쫒아 버"
마리가 제비 둥지를 덮치려는 참이었습니다. 흥부는 얼른 구렁이를 쫒아 버 부       부 부 부 ..부 기  이    다해다그   .습  니습  부부부 물가는"이 이 니 니게부이부부었   부니 니부   를부  다 나니니니  습니.부 부습  부  .니 다부  이  부 부" 니는나나 부가.부어   는 을이 어  .  로부 . 도니부.이다.다  .  부  을          나. 부 .   이  부 해 .부 을 을 니    니부  니   다.니.  .니부니  을 .니"   .부 리 .습니 ..어  니니이다렁다 부 니 니다  . 니부   ..     부부씨니니 . 히게       부서  부부. 게이.부습부 .이  다니 부리이  어부부부다  . 부부는 부니. 비 .  . . 부니니니.부  니이   습비니이리부 부..     이 다을.    부니   니이  습니 를.비 부했..   는 
Epoch 2/60
724/724 [==============================] - 5s 7ms/step - loss: 4.2354

----- Generating text after Epoch: 1
----- Generating with seed: "주었습니다. 집도 돈도 패물도 모두 뺏기고 실컷 두들겨 맞아 앓고 있는 "
주었습니다. 집도 돈도 패물도 모두 뺏기고 실컷 두들겨 맞아 앓고 있는  나"을그  다은           을  습  .을    어. 습        은서  습. .        는 살   습     었      다   질습   을아 을  . 습습 니     을       습" 습 부   습 " 운 " 습    습  

 재산은 형인 놀부가 몽땅 차지해서 동생인 흥부는 가난하게 살아야 했습니다 흥 다   흥 부, . 아 다  아.  .자 .다흥, 습  박가 놀   습 쩔오     쳤    가 다는 가못을 니 리 부 아? 다,흥는렸를 " 오이 흥 다 물 다.은아  아다다이 니"아 가습가 그  놀  아다라 가 도가  아나 는 가   습  흥  아  은아 아  아다  물시다 아부   가 와습 .와, 아라흥.아 아허 아  ,아 리 아이 철흥 아아 이 습   이  이다 다서 다. 다다  부 은  다이 다 습  가고 다.. 다    습 은는  마 마아운 니 다서 다  아 .이은  가다흥 흥이 흥 아가  는는 습는 다습 제 라 부다 아에 가 게습이가  아 와요아 아 습아 다" 자아흥아이 도 흥 날 흥아 다아 다습러 다 나아 "아 이 아습 가  습다. 아 외  흥 이다습다 다 이 부  마   다는 는 부아 
Epoch 14/60
724/724 [==============================] - 5s 7ms/step - loss: 3.8679

----- Generating text after Epoch: 13
----- Generating with seed: "가 되었습니다. 이것을 알게 된 놀부는 흥부를 찾아가 호통을 쳤습니다. "
가 되었습니다. 이것을 알게 된 놀부는 흥부를 찾아가 호통을 쳤습니다. 니 로    가 습"니다 서 다이    다다 흥 다.습부을더다,는  는 비 해    어  작다 로 다나  영  습이 아 놀는   비는부 그 다 흥 게 다모 넣  다" 을고늘  다게   .  다다 다 나 은  다 그 둥 는  부 다.다  정 다부  니다같는라다다살 로 다비 습녀 다지는는 다주 로나다  는 다은 다는 지습 습이?은썩 와릴비   져 부다아이 난부다나 물 패와으 는는 는 다수니 못다흥이 다니박 은 고 다다는 는 부   다다 비 는외부은 세면 니 르부 부 이 통. 놀   인고.다다더는 니다니 박부   르다다 운심 다 이다아 다떨 이이  다  비다  .  이  다은더  고다

참이었습니다. 흥부는 얼른 구렁이를 쫒아 버렸습니다. 떨어진 아기 제비는 흥부는 아리가 아이 "이 박이 아이 "아가 아이  흥부  아부  아   이  이   부  아이  부  아 부 살부  놀부  아이  아   이 " 이  이  부는 물이  리부  아이  부가 아아  아부는 흥부이 물리가 흥부는 아리가 그이  아이  아이 흥비이 흥부는 흥부는 아부  흥부가 그이  아이 살아  아이  아이 놀부  아비  리이 "리가 흥부와 흥부와 흥부는 놀부는 아부는 흥부  아이  나아  흥부  아와  아   부는 흥부마 아리와 아리르 아리가 놀부  아이  부  아이  으이 "나  아부  아이  아  나험 " 부  아이 아해  렸가 다아  아을  아이 흥이는 흥부  아나  아이 쳤부나 아이 "이 날  이 "  부  이이 물다  어이  아  "이 아부습니다. "이이박 이은 살부  아   아습 다.
Epoch 27/60
724/724 [==============================] - 5s 7ms/step - loss: 2.7354

----- Generating text after Epoch: 26
----- Generating with seed: " 떨어뜨려 주었습니다. 그것은 박 씨였습니다. 흥부는 제비가 준 박 씨를"
 떨어뜨려 주었습니다. 그것은 박 씨였습니다. 흥부는 제비가 준 박 씨를 흥부는 있고 아고 박  을 습니다. "허 그고  아 가  나   고  아 가   서 다. 흥부  나  부는 흥부  할부  다  나을 "  부  니다. 흥부는 흥부는 아  니다습 다. 그리가 아  고  라        다   음   이 흥부는 흥부  부  니다. 흥부  렸습니다. "허  부  이  다라 흥부  마 가 습니다. 흥부  이 가  부  라 박 다. "허 주나  다는 흥부  아  부  아  다  되 습니다. "허 박었 니다. 흥부는 아  니다. 흥부  으  다  고  다을 나부  아  부   다  . 습니다. "리가 나이 같은 제비  고  외  렁보 다. 흥부  되 씨 

날아 다니게 되었습니다. 그리고 가을이 되자 다른 제비들과 함께 남쪽으로다 다  가   가 . 오    부    는     은 쳤 .      습 아        놀       아이  .    으     아  이 아      아  박다 와  아  런 나   부   다  었이    부  " 는  아 렁  부 커부 만부    놀   놀 아 부   아   "   와    아        가 이와 부     리       "이"! 다아 는고    부 이 아    어  .  부     주다  부니부    놀부니    나와        부아   이  부       는     되.     벌이 다  아  아  다  부  살  다가  와 아아   가, 부 "   부리  리   다  살   부 흥    이  다      부  부리        리 아부  가       놀  다부    이 나 .
Epoch 40/60
724/724 [==============================] - 5s 7ms/step - loss: 3.7489

----- Generating text after Epoch: 39
----- Generating with seed: "싹이 트고 덩굴이 뻗더니, 커다란 박이 주렁주렁 달렸습니다. "허, 그 "
싹이 트고 덩굴이 뻗더니, 커다란 박이 주렁주렁 달렸습니다. "허, 그      이  .         니.    .    을    는   을 다.  제 와 고  니 이    ,    가    니 다. 심 다   다    ,   요  .     도            니다   마 로   .다    "            이               물  .    아          "            .    "   와    . .   부         놀다.  부   는 를           " " .            " 놀  이가 주 무    습   이는    다    비     다다 다 부           다 다      부      

신령님께 바쳐야 하느니라, 어험!" 놀부와 부인은 벌벌 떨며 돈과 패물을 " 이 흥부는 제이가 " 박 가 갈 부 훙 갈 박 씨 히 부   나 "  부는 "이이 살리는 찾이가 " 부   무  부와 웬리이 깨 을 " 부  는  이   아는 "이뜨  아 는 이 부 게 이 박 나 가 쳤인  부와  아   가 금 가 흥부는 흥부는 " 이 기 박   는 제이가 흥부는 흥부는 흥부는 " 이 " 박 가 박 가 박 부  이이 "이 웬는 박이  부는 물기을 " 이 갈 병 하나  벌을 " 이 가 갈 가 흥부는 흥이운  이서 "아가 흥부는 아이러  아  웬  이 영 씨 가 가 가 박 이 " " 이 가 병 주  리부  아이  아 부  부는 그뱃러 " 이 나 가 알 씨 쳤이  부는 찾이서 " 가 갔 박 " 가 갈 이 흥부는 아아이 " 철 가 욕 욕 는 갈부는 "이하 " 박이  부는 흥이와 " 갚부는 그
Epoch 53/60
724/724 [==============================] - 5s 7ms/step - loss: 2.0355

----- Generating text after Epoch: 52
----- Generating with seed: "산은 형인 놀부가 몽땅 차지해서 동생인 흥부는 가난하게 살아야 했습니다."
산은 형인 놀부가 몽땅 차지해서 동생인 흥부는 가난하게 살아야 했습니다. "기 , 은 박 을 부을 "리는 살도 게 을 살 는 제비  아을 "은 박어 , 주는 떨리  깨이 "리  부을 "인 쩍 마 는 떨이와 아을는 놀부  아을 "아 차고  부을 "리와 "을 박리을 그리는 흥리와 아인  고을 "아 게 나 박  이  이 갈물는 흥부는 아을 "고 , 져 영 박 만고는 놀부는 놀부는 흥께는 아타 "아을 물을 " 팔습니다. "형는 놀부는 아리는 흥부는 아리는 아리는 아은  물을 "른 "고 박을 패고  아 는 김이  아을 살아 는 박을 "가 패부는 흥부는 놀이는 아인  아운  아 게 마서는 살부는 흥부는 아운와 물을 "아 뺏다  산인 "아 " 박서 굳을  어을 

In [ ]:
#######################################

In [15]:
path = '어린왕자.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)

print('corpus length:', len(text))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 0: invalid start byte

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []

In [ ]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])